In [1]:
import pandas as pd
import numpy as np

ground_truth = pd.read_csv('data/partitions.txt',header=None, sep = ' ')
cluster1 = pd.read_csv('data/clustering_1.txt',header=None, sep = ' ')
cluster2 = pd.read_csv('data/clustering_2.txt',header=None, sep = ' ')
cluster3 = pd.read_csv('data/clustering_3.txt',header=None, sep = ' ')
cluster4 = pd.read_csv('data/clustering_4.txt',header=None, sep = ' ')
cluster5 = pd.read_csv('data/clustering_5.txt',header=None, sep = ' ')

In [43]:
from operator import mul    # or mul=lambda x,y:x*y
from fractions import Fraction
from functools import reduce

def entropy(data):
    n = len(data)
    value = 0
    for i in np.unique(data):
        p = len(data[data == i]) / n
        value += - p * np.log2(p)
    return value
    
def NMI(C, T):
    score = 0
    n = len(C)
    H_C = entropy(C[1])
    H_T = entropy(T[1])
    
    C_Cluster = np.unique(C[1])
    T_Cluster = np.unique(T[1])
    pC = [len(C[C[1] == i]) / n for i in C_Cluster]
    pT = [len(T[T[1] == i]) / n for i in T_Cluster]
    for i in range(0, len(pC)):
        Ci = C[C[1] == C_Cluster[i]]
        Ti = T.loc[Ci.index]
        for j in range(0, len(pT)):
            p_ij = len(Ti[Ti[1] == T_Cluster[j]]) / n
            if p_ij > 0:
                score += p_ij * np.log2(p_ij / (pC[i] * pT[j]))
    
    score /= np.sqrt(H_C * H_T)
    
    return score

def nCk(n,k):
    return int( reduce(mul, (Fraction(n-i, i+1) for i in range(k)), 1) )

def Jaccard(C, T):
    n = len(C)
    C_Cluster = np.unique(C[1])
    T_Cluster = np.unique(T[1])
    
    TP = 0
    for i in C_Cluster:
        Ci = C[C[1] == i]
        Ti = T.loc[Ci.index]
        for j in T_Cluster:
            n_ij = len(Ti[Ti[1] == j])
            TP += n_ij ** 2
    TP = (TP - n) / 2
    
    FN = 0
    for j in T_Cluster:
        m_j = len(T[T[1] == j])
        FN += nCk(m_j, 2)
    FN -= TP
    
    FP = 0
    for i in C_Cluster:
        n_i = len(C[C[1] == i])
        FP += nCk(n_i, 2)
    FP -= TP
    
    return TP / (TP + FN + FP)

In [52]:
print("%1.4f %1.4f"%(NMI(cluster1, ground_truth), Jaccard(cluster1, ground_truth)))
print("%1.4f %1.4f"%(NMI(cluster2, ground_truth), Jaccard(cluster2, ground_truth)))
print("%1.4f %1.4f"%(NMI(cluster3, ground_truth), Jaccard(cluster3, ground_truth)))
print("%1.4f %1.4f"%(NMI(cluster4, ground_truth), Jaccard(cluster4, ground_truth)))
print("%1.4f %1.4f"%(NMI(cluster5, ground_truth), Jaccard(cluster5, ground_truth)))

0.8896 0.9117
0.6456 0.6795
0.3915 0.4649
0.7678 0.8006
0.7612 0.5976
